In [9]:
import pandas as pd
import numpy as np
import utilitaires_810 as u810
import gc

DEFAULT_CATEGORY_MEAN_FREQ = 0.03
DEFAULT_EMPTY_FEAT_TRESHOLD_FOR_ROW = 0.4

# PREVIOUS APP

In [10]:
og = pd.read_csv('./input_data/previous_application.csv')

In [11]:
og['NAME_CONTRACT_STATUS'].value_counts()

Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: NAME_CONTRACT_STATUS, dtype: int64

In [15]:
def previous_applications(num_rows=None, nan_as_category=False, path='./input_data/',
                          encoding_treshold=DEFAULT_CATEGORY_MEAN_FREQ,
                          nan_treshold=DEFAULT_EMPTY_FEAT_TRESHOLD_FOR_ROW):
    features_torem = ['WEEKDAY_APPR_PROCESS_START',
                      'NAME_TYPE_SUITE',  # On ne doit pas prendre en compte cet element
                      'NAME_GOODS_CATEGORY',  # Peu importe ce que la personne a achete precedemment
                      'CODE_REJECT_REASON',  # Apporte peu d'info mais ajoute 9 features + aggregation
                      'DAYS_DECISION',
                      # I dont see the connection between a decision we made and a demand they make (logreg doest care (212) while RF does (35) + highly correlated to INSTALL_DAYS_ENTRY_PAYMENT ...
                      'CHANNEL_TYPE',
                      # Peu d'interet de connaitre le canal d'acquisition, alors que ce va nous rajouter des colonnes en dummyfiant
                      'NAME_CONTRACT_TYPE',  # Cree des colonnes additionnelles tres correlees
                      'NAME_PRODUCT_TYPE',  # Je ne vois pas l'interet et ca ajoute des colonnes
                      'NAME_SELLER_INDUSTRY',  # Peu d'interet. Patronizing
                      'FLAG_LAST_APPL_PER_CONTRACT', # Peu d'interet alors que ca cree beaucoup de colonnes
                      'NAME_CASH_LOAN_PURPOSE', # TMI
                      'NAME_PAYMENT_TYPE', # tmi
                      'NAME_CLIENT_TYPE', # Ne doit pas entrer en ligne de compte pour accorder un pret
                      'NAME_PORTFOLIO',
                      'PRODUCT_COMBINATION', # Beaucoup de categories et peu d'infos
                      'HOUR_APPR_PROCESS_START'
                      ]
    prev = pd.read_csv(path + 'previous_application.csv', nrows=num_rows)
    prev = u810.remove_too_nany_observations(prev)
    install = pd.read_csv(path + 'installments_payments.csv', nrows=num_rows)
    install = u810.remove_too_nany_observations(install)

    # Pas d'interet de conserver les demarches abandonnees, elles faussent nos indicateurs
    prev = prev[(prev['NAME_CONTRACT_STATUS']!='Canceled') & (prev['NAME_CONTRACT_STATUS']!='Unused offer')]

    prev.loc[:, 'YIELD_GROUP_HIGH'] = 0
    prev.loc[prev['NAME_YIELD_GROUP'] == 'high', 'YIELD_GROUP_HIGH'] = 1
    prev.drop(columns=['NAME_YIELD_GROUP'], inplace=True)

    # Feature engineering: ratios and difference
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT'] / prev['AMT_ANNUITY']
    prev['DOWN_PAYMENT_TO_CREDIT'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT']

    # Interest ratio on previous application (simplified)
    total_payment = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    prev['SIMPLE_INTERESTS'] = (total_payment / prev['AMT_CREDIT'] - 1) / prev['CNT_PAYMENT']




    prev.loc[:, 'HAS_REFUSED'] = 0
    prev.loc[prev['NAME_CONTRACT_STATUS'] == 'Refused', 'HAS_REFUSED'] = 1
    prev.loc[:, 'HAS_ACTIVE'] = 0
    prev.loc[(prev['NAME_CONTRACT_STATUS'] == 'Approved') & (prev['DAYS_LAST_DUE'] == 365243), 'HAS_ACTIVE'] = 1



    # CREDITS ACTIFS
    # Active loans - approved and not complete yet (last_due 365243) - valeur speciale !
    approved = prev[prev['NAME_CONTRACT_STATUS'] == "Approved"]
    active_df = approved[approved['DAYS_LAST_DUE'] == 365243]
    # Find how much was already payed in active loans (using installments csv)
    active_pay = install[install['SK_ID_PREV'].isin(active_df['SK_ID_PREV'])]

    active_pay_agg = active_pay.groupby('SK_ID_PREV')[['AMT_INSTALMENT', 'AMT_PAYMENT']].sum()
    active_pay_agg.reset_index(inplace=True)
    active_pay_agg['INSTALMENT_PAYMENT_DIFF'] = active_pay_agg['AMT_INSTALMENT'] - active_pay_agg['AMT_PAYMENT']

    # Merge with active_df
    active_df = active_df.merge(active_pay_agg, on='SK_ID_PREV', how='left')

    active_df['REMAINING_DEBT'] = active_df['AMT_CREDIT'] - active_df['AMT_PAYMENT']

    bureau_active_agg = {
        'REMAINING_DEBT': ['sum'],
        'AMT_INSTALMENT': ['sum'],
    }
    active_agg_df = active_df.groupby('SK_ID_CURR').agg({**bureau_active_agg})
    active_agg_df.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg_df.columns.tolist()])

    # active_agg_df['TOTAL_REPAYMENT_RATIO'] = active_agg_df['ACTIVE_AMT_PAYMENT_SUM'] / active_agg_df[
    #     'ACTIVE_AMT_CREDIT_SUM']
    # active_agg_df.drop(columns=['ACTIVE_AMT_PAYMENT_SUM', 'ACTIVE_AMT_CREDIT_SUM'], inplace=True)
    del active_pay, active_pay_agg, active_df;
    gc.collect()


    # CREDITS REFUSED
    # Aggregations for approved and refused loans
    refused = prev[prev['NAME_CONTRACT_STATUS'] == "Refused"]
    bureau_refused_agg = {
        'APP_CREDIT_PERC': ['mean'],
        'DAYS_DECISION': ['mean']
    }
    refused_agg_df = refused.groupby('SK_ID_CURR').agg({**bureau_refused_agg})
    refused_agg_df.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg_df.columns.tolist()])
    del refused; gc.collect()


    # CREDITS APPROVED
    bureau_approved_agg = {
        'AMT_DOWN_PAYMENT': ['sum']
    }
    approved_agg_df = approved.groupby('SK_ID_CURR').agg({**bureau_approved_agg})
    approved_agg_df.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg_df.columns.tolist()])
    del approved; gc.collect()


    # LATE PAYMENTS
    # Get the SK_ID_PREV for loans with late payments (days past due)
    install['LATE_PAYMENT'] = install['DAYS_ENTRY_PAYMENT'] - install['DAYS_INSTALMENT']
    install['LATE_PAYMENT'] = install['LATE_PAYMENT'].apply(lambda x: 1 if x > 0 else 0)
    dpd_id = install[install['LATE_PAYMENT'] > 0]['SK_ID_PREV'].unique()

    prev.loc[:, 'HAD_LATE_PAYMENTS'] = 0
    prev.loc[prev['SK_ID_PREV'].isin(dpd_id), 'HAD_LATE_PAYMENTS'] = 1


    # Change 365.243 values to nan (missing)
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    # Days last due difference (scheduled x done)
    prev['DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']

    prev = prev[[col for col in prev.columns if col not in features_torem]]

    prev_aggregation = {
        'CREDIT_TO_ANNUITY_RATIO': ['mean'],
        'SIMPLE_INTERESTS': ['mean'],
        'DOWN_PAYMENT_TO_CREDIT': ['mean'],
        'DAYS_LAST_DUE_DIFF': ['mean'],
        'YIELD_GROUP_HIGH': ['sum'],
        'HAS_REFUSED':['sum'],
        'HAS_ACTIVE':['sum'],
        'HAD_LATE_PAYMENTS':['mean']
    }

    prev = prev.groupby('SK_ID_CURR').agg({**prev_aggregation})  # , **cat_aggregations
    prev.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev.columns.tolist()])


    # MERGE ALL DF
    prev = prev.merge(active_agg_df, how='left', on='SK_ID_CURR')
    del active_agg_df;
    gc.collect()
    prev = prev.merge(refused_agg_df, how='left', on='SK_ID_CURR')
    del refused_agg_df;
    gc.collect()
    prev = prev.merge(approved_agg_df, how='left', on='SK_ID_CURR')
    del approved_agg_df;
    gc.collect()




    prev.loc[prev['PREV_HAS_ACTIVE_SUM'] == 0, 'ACTIVE_REMAINING_DEBT_SUM'] = 0
    prev.loc[prev['PREV_HAS_ACTIVE_SUM'] == 0, 'ACTIVE_AMT_INSTALMENT_SUM'] = 0

    prev.loc[prev['PREV_HAS_REFUSED_SUM'] == 0, 'REFUSED_APP_CREDIT_PERC_MEAN'] = 0
    prev.loc[prev['PREV_HAS_REFUSED_SUM'] == 0, 'REFUSED_DAYS_DECISION_MEAN'] = 0

    # prev.drop(columns = ['PREV_HAS_ACTIVE_SUM'], inplace=True)
    # Comme on a peut etre fait des divisions par 0 ...
    prev = u810.replace_infinite_by_nan(df=prev, list_new_columns=prev.columns.tolist())
    return prev

In [16]:
pv=previous_applications()

Forme initiale du Dataframe :  (1670214, 37)


/home/maximorose/git_repositories/oc-ds-p07/utilitaires_810.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df.drop(columns=['taux_nan'], inplace=True)


Forme du Dataframe apres traitement :  (1234578, 37)
Forme initiale du Dataframe :  (13605401, 8)
Forme du Dataframe apres traitement :  (13605401, 8)


In [18]:
pv.isna().sum()[pv.isna().sum()!=0]

PREV_CREDIT_TO_ANNUITY_RATIO_MEAN        23
PREV_SIMPLE_INTERESTS_MEAN           104300
PREV_DOWN_PAYMENT_TO_CREDIT_MEAN      20396
PREV_DAYS_LAST_DUE_DIFF_MEAN          26045
REFUSED_APP_CREDIT_PERC_MEAN              6
APPROVED_AMT_DOWN_PAYMENT_SUM           626
dtype: int64

# BUREAU

In [21]:
bur = pd.read_csv('./input_data/bureau.csv')

In [22]:
bur['CREDIT_ACTIVE'].value_counts()

Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64